In [ ]:
import pandas as pd 
small_resnet_df = pd.read_csv("logging/small_resnet/bs=32_dataset=full_ideal_firstNeurons=64_augmented.csv")
small_no_dropout_df = pd.read_csv("logging/small_resnet/bs=64_dataset=full_ideal_firstNeurons=64_augmented__no_dropout.csv")
resnet50_df = pd.read_csv("logging/resnet50/bs=32_dataset=full_ideal_firstNeurons=64_augmented.csv")
transfer_df = pd.read_csv("logging/resnet50_pretrained/bs=32_dataset=full_ideal_firstNeurons=transfer_learn.csv")

In [ ]:
def get_max_values(input_df):
    min_idx = input_df["val_mean_absolute_error"].idxmin()
    print(min_idx+1)
    print(input_df.loc[min_idx])


    min_idx = input_df["val_mean_absolute_percentage_error"].idxmin()
    print(min_idx+1)
    print(input_df.loc[min_idx])

    max_idx = input_df["val_r_squared"].idxmax()
    print(max_idx+1)
    print(input_df.loc[max_idx])

In [ ]:
get_max_values(small_resnet_df)

In [ ]:
get_max_values(resnet50_df)

In [ ]:
get_max_values(transfer_df)

In [ ]:
import matplotlib.pyplot as plt  


def plot_loss(data_df, loss_path, percent_path):
    epochs = data_df["epoch"]
    mean_absolute_error = data_df["mean_absolute_error"]
    mean_absolute_percentage_error = data_df["mean_absolute_percentage_error"]
    val_mean_absolute_error = data_df["val_mean_absolute_error"]
    val_mean_absolute_percentage_error = data_df["val_mean_absolute_percentage_error"]
    
    plt.plot(epochs, mean_absolute_error, val_mean_absolute_error)
    plt.xlabel('epochs')
    plt.ylabel('Loss (Mean Absolute Error) (PDFF)')
    plt.ylim(top=2.6)
    plt.title('Loss vs Epochs')    
    plt.legend(['train loss', 'validation loss'])
    plt.savefig(loss_path)
    plt.show()    
    plt.plot(epochs, mean_absolute_percentage_error, val_mean_absolute_percentage_error, label=['train MAPE, validation MAPE'])
    plt.xlabel('epochs')
    plt.ylabel('Mean Absolute Percentage Error (%)')
    plt.ylim(top=50)
    plt.title('Mean Absolute Percentage Error vs Epochs')
    plt.legend(['train % error', 'validation % error'])
    plt.savefig(percent_path)
    plt.show()
    

In [ ]:
plot_loss(small_resnet_df, loss_path = "plots/small_resnet_bs_32_loss.png", percent_path = "plots/small_resnet_bs_32_percent.png")

In [ ]:
plot_loss(small_no_dropout_df, loss_path="plots/resnet20_bs_64_no_drop_loss.png",percent_path="plots/resnet20_bs_64_no_drop_percent.png" )

In [ ]:
plot_loss(resnet50_df, loss_path="plots/resnet50_bs_64_loss.png", percent_path="plots/resnet50_bs_64_percent.png")

In [ ]:
plot_loss(transfer_df, loss_path="plots/transfer_bs_32_loss.png", percent_path="plots/transfer_bs_32_percent.png")

# Extract info from logs

In [ ]:
with open("slurm-2007264.out") as file:
    lines = [line.rstrip() for line in file]
len(lines)

In [ ]:

line = lines[0]
i = 0
log_indices = []
for line in lines:

    if "Epoch" in line:
        log_indices.append(i-1)
    i += 1
log_indices.pop(0)
print(len(log_indices))

In [ ]:
metric_names = ["epoch","loss","lr","mean_absolute_error","mean_absolute_percentage_error","r_squared","val_loss","val_mean_absolute_error","val_mean_absolute_percentage_error","val_r_squared"]
log_df = pd.DataFrame(columns=metric_names)

log_metric_names = ["loss", "mean_absolute_error", "mean_absolute_error", "mean_absolute_percentage_error", "r_squared", "val_loss", "val_mean_absolute_error", "val_mean_absolute_percentage_error", "val_r_squared", "lr"]
line_dict = dict()

epoch = 1

for index in log_indices:
    line = lines[index]

    if epoch > 17:
        log_metric_names = ["loss", "mean_absolute_error", "mean_absolute_error", "mean_absolute_percentage_error", "r_squared_1", "val_loss", "val_mean_absolute_error", "val_mean_absolute_percentage_error", "val_r_squared_1"]

    for j in range(0, len(log_metric_names)): #Handle                 

        start_metric_name = log_metric_names[j] + ": "
        start_index = line.find(start_metric_name) + len(start_metric_name)
        if j == len(log_metric_names) - 1:
            end_index = len(line)
        else:
            end_metric_name = "- " + log_metric_names[j+1]    
            end_index =  line.find(end_metric_name)

        metric_val = line[start_index : end_index]    
        metric_key = log_metric_names[j]
        if metric_key[-2:] == "_1":
            metric_key = metric_key[:-2]
        
        line_dict[metric_key] = metric_val
        line_dict["epoch"] = epoch
    
    line_df = pd.DataFrame(line_dict, index=[0])
    log_df = pd.concat([log_df, line_df])

    epoch += 1

log_df.loc[((log_df["epoch"] >= 40) & (log_df["epoch"] < 65)), 'lr'] = 0.00001
log_df.loc[((log_df["epoch"] >= 65) & (log_df["epoch"] < 75)), 'lr'] = 0.000001

log_df.head()

In [ ]:
log_df.to_csv("logging/resnet50_pretrained/bs=32_dataset=full_ideal_firstNeurons=transfer_learn.csv")